## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [150]:
# Load up store_income_data.csv
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
import datetime


In [151]:
df = pd.read_csv("store_income_data_task.csv")
df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [152]:
countries = df["country"].unique()
number_of_countries = df["country"].nunique()
print(countries)
print(number_of_countries)


['United States/' 'Britain' ' United States' 'Britain/' ' United Kingdom'
 'U.K.' 'SA ' 'U.K/' 'America' 'United Kingdom' nan 'united states'
 ' S.A.' 'England ' 'UK' 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'U.K '
 'America/' 'SA.' 'S.A. ' 'u.k' 'uk' ' ' 'UK.' 'England/' 'england'
 ' Britain' 'united states of america' 'UK/' 'SA/' 'SA' 'England.'
 'UNITED KINGDOM' 'America.' 'S.A..' 's.a.' ' U.K'
 ' United States of America' 'Britain ' 'England' ' SA'
 'United States of America.' 'United States of America/' 'United States.'
 's. africasouth africa' ' England' 'United Kingdom '
 'United States of America ' ' UK' 'united kingdom' 'AMERICA' 'America '
 'UNITED STATES OF AMERICA' ' S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' '/' 'United Kingdom.' 'United States'
 ' America' 'UNITED STATES' 'sa' 'United States of America' 'UK '
 'United States ' 'S. AfricaSouth Africa/' 'S.A.' 'United Kingdom/'
 'S. AfricaSouth Africa ' 'S. AfricaSouth Africa.' 'S. AfricaSouth Africa'
 '.

In [153]:
df["country"] = df["country"].str.lower()
df["country"] = df["country"].str.strip()
countries = df["country"].unique()
number_of_countries = df["country"].nunique()
print(countries)
print(number_of_countries)


['united states/' 'britain' 'united states' 'britain/' 'united kingdom'
 'u.k.' 'sa' 'u.k/' 'america' nan 's.a.' 'england' 'uk' 's.a./' 'u.k'
 'america/' 'sa.' '' 'uk.' 'england/' 'united states of america' 'uk/'
 'sa/' 'england.' 'america.' 's.a..' 'united states of america.'
 'united states of america/' 'united states.' 's. africasouth africa'
 'britain.' '/' 'united kingdom.' 's. africasouth africa/'
 'united kingdom/' 's. africasouth africa.' '.']
36


In [154]:
matches = fuzzywuzzy.process.extract("uk", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('uk', 100),
 ('uk.', 100),
 ('uk/', 100),
 ('u.k.', 40),
 ('u.k/', 40),
 ('u.k', 40),
 ('united states/', 13),
 ('united states', 13),
 ('united states.', 13),
 ('united kingdom', 12)]

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [155]:
# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    print("All done!")

In [156]:
replace_matches_in_column(df=df,column="country",string_to_match = "united kingdom")
replace_matches_in_column(df=df,column="country",string_to_match = "united states")
replace_matches_in_column(df=df,column="country",string_to_match = "south africa")

All done!
All done!
All done!


In [157]:
countries = df["country"].unique()
no_of_countries = df["country"].nunique()
print(countries)
print(no_of_countries)

['united states' 'britain' 'britain/' 'united kingdom' 'u.k.' 'sa' 'u.k/'
 'america' nan 's.a.' 'england' 'uk' 's.a./' 'u.k' 'america/' 'sa.' ''
 'uk.' 'england/' 'united states of america' 'uk/' 'sa/' 'england.'
 'america.' 's.a..' 'united states of america.'
 'united states of america/' 's. africasouth africa' 'britain.' '/'
 's. africasouth africa/' 's. africasouth africa.' '.']
32


In [158]:
df["country"] = df["country"].str.strip("/")
df["country"] = df["country"].str.strip(".")
countries = df["country"].unique()
no_of_countries = df["country"].nunique()
print(countries)
print(no_of_countries)

['united states' 'britain' 'united kingdom' 'u.k' 'sa' 'america' nan 's.a'
 'england' 'uk' '' 'united states of america' 's. africasouth africa']
12


In [159]:
df.replace("britain","united kingdom",inplace=True)
df.replace("uk","united kingdom",inplace=True)
df.replace("u.k","united kingdom",inplace=True)
df.replace("england","united kingdom",inplace=True)
df.replace("america","united states",inplace=True)
df.replace("united states of america","united states",inplace=True)
df.replace("sa","south africa",inplace=True)
df.replace("s.a","south africa",inplace=True)
df.replace("s. africasouth africa","south africa",inplace=True)

countries = df["country"].unique()
print(countries)

['united states' 'united kingdom' 'south africa' nan '']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [167]:
from datetime import date
df["date_parsed"] = pd.to_datetime(df["date_measured"], format="%d-%m-%Y")
df["days_ago"]= datetime.date.today() - df["date_parsed"].dt.date
df.head()

,id,store_name,store_email,department,income,date_measured,country,date_parsed,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states,2006-02-04,"6656 days, 0:00:00"
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united kingdom,2006-01-04,"6687 days, 0:00:00"
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states,2003-09-12,"7532 days, 0:00:00"
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united kingdom,2006-05-08,"6563 days, 0:00:00"
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united kingdom,1973-01-21,"18723 days, 0:00:00"
